Niall Carbery 22380966. When asked for the expectation value we are concerned with measurement outcomes, and
not building an evolution operator out of the Hamiltonian. Keep this in mind throughout
the project.

In [125]:
import numpy as np
import scipy.linalg

<b>Consider the following two qubit hamiltonian $\hat H_a$:
$\hat H_a$ = Z0 ⊗ Z1
where Z0, Z1 represent the Pauli-Z operator applied to qubits 0 and 1 respectively. Given an arbitrary
two qubit state vector |ψ⟩, we can compute the expectation value of the hamiltonian as follows:
⟨ψ|$\hat H_a$ |ψ⟩
If we can only perform measurements on the qubits in the Z-basis, {|0⟩, |1⟩}, how can we use the
measurement outcomes on the two qubits to calculate the expectation value of $\hat H_a$?</b>

Our quibits are in an initial state of the quibits and are a superposition of the basis states {|0>, |1>}. Measuring the quibits over the Z operater we get the eigenvalues of +1 for |0> and -1 for |1>.

<p>
For the two quibit system the Z expectation value is given by the number counts of |00>, |01>, |10> and |11>. We can estimate the expectation value by measuring the two quibits the expectation value of $\hat H_a$ = Number of |00> + |11> - |01> - |10>.

In [126]:
# Define the Pauli Matrices

def dagger(state):
    return np.transpose(np.conj(state))

pauli_I = np.array([[1,0],[0,1]])
pauli_X = np.array([[0,1],[1,0]])
pauli_Y = np.array([[0,-1j],[1j,0]])
pauli_Z = np.array([[1,0],[0,-1]])

pauli_matrices = [pauli_I, pauli_X, pauli_Y, pauli_Z]

# Define Hadamard Gate
hadamard = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])

# Define Phase gate
s_phase  = np.array([[1,0],[0,1j]])

In [127]:
# Two quibits
hama = np.kron(pauli_X, pauli_X)

def find_pauli_coeffs_2qubit(ham):
    ham_pauli_coeffs = {}
    for indexA, current_pauli_A in enumerate(pauli_matrices):
        for indexB, current_pauli_B in enumerate(pauli_matrices):
            current_pauli_string   = np.kron(current_pauli_A, current_pauli_B)
            current_ham_projection = np.real((1/2**2)*np.trace(ham@current_pauli_string))
            if current_ham_projection != 0.0: #Let's only include non-zero coefficients in our expansion
                ham_pauli_coeffs.update({f"{indexA}{indexB}": current_ham_projection})
    return ham_pauli_coeffs


ham_tot_pauli_decomp = find_pauli_coeffs_2qubit(hama)


def basis_transform_2qubit(pauli_string):
    system_A_pauli = int(pauli_string[0])
    system_B_pauli = int(pauli_string[1])
    
    system_A_transform = None
    system_B_transform = None
    
    if system_A_pauli  == 0:
        system_A_transform = pauli_I
    elif system_A_pauli == 1:
        system_A_transform = hadamard
    elif system_A_pauli == 2:
        system_A_transform = hadamard@dagger(s_phase)
    elif system_A_pauli == 3:
        system_A_transform = pauli_I
    
    if system_B_pauli == 0:
        system_B_transform = pauli_I
    elif system_B_pauli == 1:
        system_B_transform = hadamard
    elif system_B_pauli == 2:
        system_B_transform = hadamard@dagger(s_phase)
    elif system_B_pauli == 3:
        system_B_transform = pauli_I
        
    return np.kron(system_A_transform, system_B_transform)


def expectation_from_decomp(ham_pauli_decomp, state):
    
    total_expectation = 0.0
    for current_key in ham_pauli_decomp.keys():
        current_transform  = basis_transform_2qubit(current_key)
        transformed_state  = current_transform@state
        system_A_pauli = int(current_key[0])
        system_B_pauli = int(current_key[1])
        
        if system_A_pauli == 0:
            system_A_op = pauli_I
        else:
            system_A_op = pauli_Z
            
        if system_B_pauli == 0:
            system_B_op = pauli_I
        else:
            system_B_op = pauli_Z
            
        total_expectation += ham_pauli_decomp[current_key]*dagger(transformed_state)@np.kron(system_A_op, system_B_op)@transformed_state

    return total_expectation

eigvals_tot, eigvecs_tot = scipy.linalg.eigh(hama)
print("HAM TOT EIGENVALUES AND EIGENVECTORS")
print(f"Ham tot EigenValues: {eigvals_tot}")
print(f"Ham tot EigenVectors: {eigvecs_tot}")

ground_state_tot = eigvecs_tot[:,0]
expectation_from_decomp(ham_tot_pauli_decomp, ground_state_tot)

HAM TOT EIGENVALUES AND EIGENVECTORS
Ham tot EigenValues: [-1. -1.  1.  1.]
Ham tot EigenVectors: [[ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.         -0.70710678  0.         -0.70710678]
 [ 0.          0.70710678  0.         -0.70710678]
 [-0.70710678  0.         -0.70710678  0.        ]]


-0.9999999999999998

###
<b>How can we now calculate the expecation for a hamiltonian with Pauli matrices other than Z? For
example, consider the following 3 qubit hamiltonian: $\hat H_c = X_0 \otimes Y_1 \otimes Z_2$</b>

In [128]:
def find_pauli_coeffs_3qubit(ham):
    ham_pauli_coeffs = {}
    for indexA, current_pauli_A in enumerate(pauli_matrices):
        for indexB, current_pauli_B in enumerate(pauli_matrices):
            for indexC, current_pauli_C in enumerate(pauli_matrices):
                current_pauli_string = np.kron(np.kron(current_pauli_A, current_pauli_B), current_pauli_C)
                current_ham_projection = np.real((1/2**3) * np.trace(ham @ current_pauli_string))
                if current_ham_projection != 0.0:
                    ham_pauli_coeffs.update({f"{indexA}{indexB}{indexC}": current_ham_projection})
    return ham_pauli_coeffs

def basis_transform_3qubit(pauli_string):
    system_A_pauli = int(pauli_string[0])
    system_B_pauli = int(pauli_string[1])
    system_C_pauli = int(pauli_string[2])
    
    system_A_transform = None
    system_B_transform = None
    system_C_transform = None
    
    if system_A_pauli == 0:
        system_A_transform = pauli_I
    elif system_A_pauli == 1:
        system_A_transform = hadamard
    elif system_A_pauli == 2:
        system_A_transform = hadamard @ dagger(s_phase)
    elif system_A_pauli == 3:
        system_A_transform = pauli_I
    
    if system_B_pauli == 0:
        system_B_transform = pauli_I
    elif system_B_pauli == 1:
        system_B_transform = hadamard
    elif system_B_pauli == 2:
        system_B_transform = hadamard @ dagger(s_phase)
    elif system_B_pauli == 3:
        system_B_transform = pauli_I
    
    if system_C_pauli == 0:
        system_C_transform = pauli_I
    elif system_C_pauli == 1:
        system_C_transform = hadamard
    elif system_C_pauli == 2:
        system_C_transform = hadamard @ dagger(s_phase)
    elif system_C_pauli == 3:
        system_C_transform = pauli_I
    
    return np.kron(np.kron(system_A_transform, system_B_transform), system_C_transform)

def expectation_from_decomp_3qubit(ham_pauli_decomp, state):
    total_expectation = 0.0
    for current_key in ham_pauli_decomp.keys():
        current_transform = basis_transform_3qubit(current_key)
        transformed_state = current_transform @ state
        system_A_pauli = int(current_key[0])
        system_B_pauli = int(current_key[1])
        system_C_pauli = int(current_key[2])
        
        if system_A_pauli == 0:
            system_A_op = pauli_I
        else:
            system_A_op = pauli_Z
            
        if system_B_pauli == 0:
            system_B_op = pauli_I
        else:
            system_B_op = pauli_Z
            
        if system_C_pauli == 0:
            system_C_op = pauli_I
        else:
            system_C_op = pauli_Z
            
        total_expectation += ham_pauli_decomp[current_key] * dagger(transformed_state) @ np.kron(np.kron(system_A_op, system_B_op), system_C_op) @ transformed_state

    return total_expectation


hama = np.kron(np.kron(pauli_X, pauli_Y), pauli_Z)
eigvals_tot, eigvecs_tot = scipy.linalg.eigh(hama)
print("HAM TOT EIGENVALUES AND EIGENVECTORS")
print(f"Ham tot EigenValues: {eigvals_tot}")
print(f"Ham tot EigenVectors: {eigvecs_tot}")

ground_state_tot = eigvecs_tot[:,0]
expectation_from_decomp(ham_tot_pauli_decomp, ground_state_tot)

HAM TOT EIGENVALUES AND EIGENVECTORS
Ham tot EigenValues: [-1. -1. -1. -1.  1.  1.  1.  1.]
Ham tot EigenVectors: [[ 0.70710678+0.j          0.        +0.j          0.        +0.j
   0.        +0.j         -0.70710678-0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.70710678j]
 [ 0.        +0.j          0.70710678+0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678+0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        -0.70710678j
   0.        +0.j          0.        +0.j          0.        -0.70710678j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.70710678j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0. 

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 8 is different from 4)

Since we can only measure in the Z- basis, we need to express the Hamiltonian in terms of the Z-basis states. Where the operaters are repersented as 
- X operator: $X = HZH$
- Y operator: $Y = SHZHS^*$



###
<b>We have some initial 3 qubit state |ϕ⟩, which could be entangled. U is a unitary transformation prior
to measurement on only qubit 0. U is chosen to implement any desired Pauli string measurement,
from which we can construct expectations</b>

In [ ]:
zero = np.array([[1], [0]])

X_gate = np.array([[0 ,1], [1, 0]])
Y_gate = np.array([[0 ,-1j], [1j, 0]])
Z_gate = np.array([[1 ,0], [0, -1]])
I_gate = np.array([[1 ,0], [0, 1]])
H_gate = (1/np.sqrt(2))*np.array([[1 ,1], [1, -1]])
S_gate = np.array([[1 ,0], [0, 1j]])

swap_gate = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
cnot_gate = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

cnot_gate13 = np.kron(I_gate, swap_gate)@np.kron(cnot_gate, I_gate)@np.kron(I_gate, swap_gate)
triple_H  = np.kron(np.kron(H_gate, H_gate), H_gate)

def dagger(state):
    return np.transpose(np.conj(state))

### Question 4

In [ ]:
def four(a, b, c, d):
    return np.kron(np.kron(np.kron(a, b), c), d)

Eh = 27.211*1.609*10**(-19)
c1 = -0.138754
c2 = -0.152989
c3 = 0.164190
c4 = 0.144579
c5 = 0.111373
c6 = 0.146726
c7 = 0.169348
c8 = -0.035353
c9 = 0.035353

# Hydrogen Hamiltonian
hydrogen = Eh*((c9*four(Y_gate,X_gate,X_gate,Y_gate))+(c9*four(X_gate,Y_gate,Y_gate,X_gate))+(c8*four(Y_gate,Y_gate,X_gate,X_gate))+ \
    (c8*four(X_gate,X_gate,Y_gate,Y_gate))+(c7*four(Z_gate,Z_gate,I_gate,I_gate))+(c6*four(Z_gate,I_gate,I_gate,Z_gate))+ \
    (c6*four(I_gate,Z_gate,Z_gate,I_gate))+(c5*four(Z_gate,I_gate,Z_gate,I_gate))+(c5*four(I_gate,Z_gate,I_gate,Z_gate))+ \
    (c4*four(I_gate,I_gate,Z_gate,Z_gate))+(c3*four(I_gate,Z_gate,I_gate,I_gate))+(c3*four(Z_gate,I_gate,I_gate,I_gate))+ \
    (c2*four(I_gate,I_gate,I_gate,Z_gate))+(c2*four(I_gate,I_gate,Z_gate,I_gate))+(c1*four(I_gate,I_gate,I_gate,I_gate)))